In [2]:
import pandas as pd
import numpy as np
from pycaret.regression import setup, compare_models, blend_models,tune_model,predict_model,get_config, finalize_model

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['font.family'] = 'Gulim'

In [14]:
from workalendar.asia import SouthKorea
import pendulum

In [38]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [39]:
train.head(3)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0


## 전처리
- 일자에서 월과 일을 분리
- 요일을 레이블 인코딩화(EDA로 요일의 중요도 순 파악)
- 월 별, 일 별 중식 석식 수요 차이 파악

In [40]:
train['월'] = pd.DatetimeIndex(train['일자']).month
test['월'] = pd.DatetimeIndex(test['일자']).month
train['주'] = pd.DatetimeIndex(train['일자']).week
test['주'] = pd.DatetimeIndex(test['일자']).week
train['일'] = pd.DatetimeIndex(train['일자']).day
test['일'] = pd.DatetimeIndex(test['일자']).day

train['출근'] = train['본사정원수']-(train['본사휴가자수']+train['본사출장자수']+train['현본사소속재택근무자수'])
train['휴가비율'] = train['본사휴가자수']/train['본사정원수']
train['출장비율'] = train['본사출장자수']/train['본사정원수']
train['야근비율'] = train['본사시간외근무명령서승인건수']/train['출근']
train['재택비율'] = train['현본사소속재택근무자수']/train['본사정원수']

test['출근'] = test['본사정원수']-(test['본사휴가자수']+test['본사출장자수']+test['현본사소속재택근무자수'])
test['휴가비율'] = test['본사휴가자수']/test['본사정원수']
test['출장비율'] = test['본사출장자수']/test['본사정원수']
test['야근비율'] = test['본사시간외근무명령서승인건수']/test['출근']
test['재택비율'] = test['현본사소속재택근무자수']/test['본사정원수']

In [41]:
weekday = {
    '월': 1,
    '화': 2,
    '수': 3,
    '목': 4,
    '금': 5
}

train['요일'] = train['요일'].map(weekday)
test['요일'] = test['요일'].map(weekday)

In [43]:
month_rank4dinner = {
    1: 11,
    2: 2,
    3: 1,
    4: 4,
    5: 7,
    6: 6,
    7: 10,
    8: 8,
    9: 5,
    10: 3,
    11: 9,
    12: 12
}
train['월(석식)'] = train['월'].map(month_rank4dinner)
test['월(석식)'] = test['월'].map(month_rank4dinner)

month_rank4lunch = {
    1: 3,
    2: 1,
    3: 2,
    4: 6,
    5: 7,
    6: 8,
    7: 10,
    8: 9,
    9: 5,
    10: 4,
    11: 11,
    12: 12
}
train['월(중식)'] = train['월'].map(month_rank4lunch)
test['월(중식)'] = test['월'].map(month_rank4lunch)

weekday_rank4dinner = {
    '월': 1,
    '화': 2,
    '수': 4,
    '목': 3,
    '금': 5,
}

weekday_rank4lunch = {
    '월': 1,
    '화': 2,
    '수': 3,
    '목': 4,
    '금': 5,
}

train['요일(석식)'] = train['요일'].map(weekday_rank4dinner)
test['요일(석식)'] = test['요일'].map(weekday_rank4dinner)

train['요일(중식)'] = train['요일'].map(weekday_rank4lunch)
test['요일(중식)'] = test['요일'].map(weekday_rank4lunch)

In [45]:
rank = pd.DataFrame(range(1,53))
week_rank_lunch = pd.pivot_table(train,values='중식계',index='주').sort_values(by='중식계').reset_index().drop('중식계',axis=1)
week_rank_dinner = pd.pivot_table(train,values='석식계',index='주').sort_values(by='석식계').reset_index().drop('석식계',axis=1)


week_rank4lunch = {}
for i in range(len(rank)):
    week_rank4lunch[week_rank_lunch['주'][i]] = rank[0][i]


week_rank4dinner = {}
for i in range(len(rank)):
    week_rank4dinner[week_rank_dinner['주'][i]] = rank[0][i]
    
    
train['주(중식)'] = train['주'].map(week_rank4lunch)
test['주(중식)'] = test['주'].map(week_rank4lunch)

train['주(석식)'] = train['주'].map(week_rank4dinner)
test['주(석식)'] = test['주'].map(week_rank4dinner)

## 공휴일 변수 생성

In [46]:
def is_holiday(date):
    holidays = list(map(str, pd.Series(np.array(SouthKorea().holidays(int(date[:4])))[:, 0])))
    
    yesterday = str(np.datetime64(date) - 1)
    tomorrow = str(np.datetime64(date) + 1)

    if tomorrow in holidays and yesterday in holidays:
        return 3 #'S'
    if tomorrow in holidays:
        return 2 # 'T'
    elif yesterday in holidays:
        return 1 #'Y'
    else : 
        return 0 #'N'

def week_of_month(x):
    dt = pendulum.parse(x)
    
    wom = dt.week_of_month
    if wom < 0:
        wom += 52
    return wom
    

df = pd.concat([train[['본사정원수', '일자']], test[['본사정원수', '일자']]])
df['년월'] = df['일자'].apply(lambda x : x[:7])
df = df[['년월', '본사정원수']].groupby(by=['년월'], as_index=False).mean()

def member_change(date):
    this_month = date[:7]
    last_month = str(np.datetime64(this_month) - 1)
    
    this_month_member = int(df[df['년월'] == this_month]['본사정원수'])
    last_month_member = int(df[df['년월'] == last_month]['본사정원수'])
    
    
    return  this_month_member - last_month_member

train['공휴일전후'] = train['일자'].apply(is_holiday)
test['공휴일전후'] = test['일자'].apply(is_holiday)

train['몇주차'] = train['일자'].apply(week_of_month)
test['몇주차'] = test['일자'].apply(week_of_month)

train = train[train['일자'] > '2016-03']
train['인원변화'] = train['일자'].apply(member_change)
test['인원변화'] = test['일자'].apply(member_change)

### train/test에서 중복 메뉴만 변수로서 사용

In [64]:
lunch_train = train.drop(columns=['본사정원수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수','일자', '요일','주', '월', '석식계', '요일(석식)','조식메뉴', '중식메뉴', '석식메뉴','주(석식)', '월(석식)'])
lunch_test = test.drop(columns=['본사정원수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수','일자', '요일','주', '월', '요일(석식)', '월(석식)', '조식메뉴', '중식메뉴','주(석식)' , '석식메뉴'])


dinner_train = train.drop(columns=['본사정원수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수','일자', '요일','주', '월', '중식계', '요일(중식)','조식메뉴', '중식메뉴', '석식메뉴','주(중식)' , '월(중식)'])
dinner_test = test.drop(columns=['본사정원수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수','일자', '요일', '주','월', '요일(중식)', '월(중식)', '조식메뉴', '중식메뉴','주(중식)' , '석식메뉴'])

In [65]:
print(lunch_train.shape)
print(lunch_test.shape)

(1187, 15)
(50, 14)


In [63]:
for c in lunch_train.columns:
    if c in ['공휴일전후', '몇주차', '인원변화', '요일(중식)', '월(중식)', '일', '주(중식)', '출근', '휴가비율',
       '출장비율', '야근비율', '재택비율', '본사출장자수', '본사휴가자수', '중식계'] :
        continue
    print(c)

본사정원수
본사시간외근무명령서승인건수
현본사소속재택근무자수


In [49]:
print(dinner_train.shape)
print(dinner_test.shape)

(1187, 18)
(50, 17)


#### 분포 확인 및 분포 조정

In [52]:
drop_index = dinner_train[dinner_train['석식계']==0].index

dinner_train.drop(drop_index, inplace=True)

print(dinner_train.shape)

(1144, 18)


# 중식 예측모델

In [57]:
num_feature = list(lunch_train.columns)
num_feature.remove('중식계')


lunch_regression_model = setup(data=lunch_train, target='중식계', train_size=0.8,n_jobs=-1,fold = 5,
                               numeric_features=num_feature,
                              numeric_imputation = 'mean',
                                normalize = True)

,Description,Value
0,session_id,2854
1,Target,중식계
2,Original Data,"(1187, 18)"
3,Missing Values,True
4,Numeric Features,17
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(949, 14)"


In [58]:
# 최고 성능 모델 선정(2개)
lunch_regression_best_models = compare_models(fold=5, n_select=2, exclude=['huber','llar','lar','par','lasso'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,78.0199,10176.6785,100.7841,0.7639,0.1206,0.0931,0.9840
gbr,Gradient Boosting Regressor,80.1106,10905.5668,104.3408,0.7469,0.1239,0.0954,0.0460
lightgbm,Light Gradient Boosting Machine,80.6746,10997.4170,104.8215,0.7451,0.1245,0.0960,0.3020
et,Extra Trees Regressor,80.6616,11034.0494,104.9681,0.7449,0.1231,0.0957,0.0940
rf,Random Forest Regressor,82.7555,11529.9728,107.2756,0.7332,0.1267,0.0983,0.1380
xgboost,Extreme Gradient Boosting,85.6823,12114.6447,110.0168,0.7191,0.1319,0.1025,0.2840
ada,AdaBoost Regressor,97.2969,14780.9340,121.4966,0.6585,0.1449,0.1175,0.0380
knn,K Neighbors Regressor,98.2602,15805.6049,125.6258,0.6341,0.1482,0.1173,0.0120
br,Bayesian Ridge,99.3961,15998.3830,126.3155,0.6297,0.1473,0.1175,0.0080
ridge,Ridge Regression,99.0928,16025.6811,126.4021,0.6290,0.1478,0.1173,0.9400


In [59]:
lunch_regression_best_models_tuned = [tune_model(i, optimize='MAE') for i in lunch_regression_best_models]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,84.1853,12682.5510,112.6168,0.6811,0.1406,0.1047
1,75.2450,9645.0303,98.2091,0.7962,0.1152,0.0883
2,74.5450,9552.7838,97.7383,0.7554,0.1163,0.0889
3,80.6499,10679.5514,103.3419,0.7960,0.1307,0.1026
4,82.9934,11443.6062,106.9748,0.7738,0.1318,0.1006
5,79.4553,10234.5618,101.1660,0.7438,0.1121,0.0896
6,79.1818,9756.4043,98.7745,0.7633,0.1109,0.0900
7,74.2802,8878.2417,94.2244,0.7686,0.1029,0.0845
Mean,78.8170,10359.0913,101.6307,0.7598,0.1201,0.0936
SD,3.5664,1140.6688,5.5030,0.0343,0.0120,0.0072


In [60]:
lunch_model = blend_models(estimator_list=lunch_regression_best_models_tuned, fold=8, optimize='MAE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,83.5860,13196.5317,114.8762,0.6682,0.1424,0.1038
1,74.9346,9626.6092,98.1153,0.7966,0.1206,0.0899
2,71.9149,8412.9382,91.7221,0.7846,0.1088,0.0854
3,77.9005,9871.7062,99.3565,0.8114,0.1243,0.0978
4,81.2116,11003.8669,104.8993,0.7824,0.1306,0.0991
5,78.6649,9894.2405,99.4698,0.7523,0.1101,0.0888
6,78.0556,9371.8646,96.8084,0.7726,0.1072,0.0878
7,76.0205,9166.4815,95.7417,0.7611,0.1040,0.0858
Mean,77.7861,10068.0298,100.1237,0.7662,0.1185,0.0923
SD,3.3903,1367.6474,6.5791,0.0410,0.0125,0.0065


In [61]:
pred = predict_model(lunch_model)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,80.7544,11206.5038,105.8608,0.7536,0.1265,0.0970


### 테스트 데이터 예측

In [50]:
submission = pd.read_csv('../data/sample_submission.csv')

In [97]:
final_model_lunch = finalize_model(lunch_model_f)
prep_pipe_lunch = get_config('prep_pipe')
prep_pipe_lunch.steps.append(['trained_model', final_model_lunch])

In [98]:
pred_lunch = prep_pipe_lunch.predict(lunch_df_test_f)

submission.iloc[:,1] = pred_lunch
submission.head()

,일자,중식계,석식계
0,2021-01-27,976.692421,199.990676
1,2021-01-28,944.873411,426.967676
2,2021-01-29,657.678285,254.959411
3,2021-02-01,1270.712029,493.758240
4,2021-02-02,1006.563512,449.174443


# 석식 예측모델

In [60]:
num_feature = list(dinner_df_f.columns)
num_feature.remove('석식계')

dinner_regression_model = setup(data=dinner_df_f, target='석식계', train_size=0.8,n_jobs=-1,fold = 5,
                                                              numeric_features=num_feature,
                                numeric_imputation = 'mean',
                                normalize = True)

,Description,Value
0,session_id,8849
1,Target,석식계
2,Original Data,"(1205, 124)"
3,Missing Values,False
4,Numeric Features,123
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(964, 117)"


In [61]:
dinner_regression_best_models = compare_models(fold=5, n_select=3, exclude=['huber','llar','lar','par','lasso'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,59.8603,7503.7998,86.5471,0.5965,1.0082,0.1245,0.0480
catboost,CatBoost Regressor,58.6499,7469.7297,86.3526,0.5960,0.9732,0.1237,0.8020
et,Extra Trees Regressor,57.3549,7644.2293,87.3583,0.5839,0.9419,0.1217,0.0800
rf,Random Forest Regressor,59.1703,7836.5536,88.3845,0.5762,0.9636,0.1282,0.0840
xgboost,Extreme Gradient Boosting,62.7262,8295.6826,90.8601,0.5561,0.9233,0.1353,0.1500
lightgbm,Light Gradient Boosting Machine,61.3606,8227.6004,90.5712,0.5553,0.9807,0.1297,0.0540
ada,AdaBoost Regressor,73.4196,9738.2801,98.6081,0.4736,1.0491,0.1575,0.0300
omp,Orthogonal Matching Pursuit,69.4732,10086.9233,100.1455,0.4618,1.1169,0.1341,0.0060
br,Bayesian Ridge,69.1182,10105.6612,100.2523,0.4601,1.1161,0.1347,0.0080
en,Elastic Net,68.0779,10305.9479,101.2548,0.4497,1.1259,0.1321,0.0060


In [62]:
dinner_regression_best_models_tuned = [tune_model(i, optimize='MAE') for i in dinner_regression_best_models]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,56.1101,6666.8092,81.6505,0.6660,1.1667,0.1218
1,59.7199,7315.6844,85.5318,0.6490,1.0847,0.1159
2,61.6354,8000.9409,89.4480,0.4918,0.7446,0.1357
3,60.4639,8053.5400,89.7415,0.6130,1.1348,0.1397
4,60.8267,7719.9373,87.8632,0.5287,0.9452,0.1163
Mean,59.7512,7551.3823,86.8470,0.5897,1.0152,0.1259
SD,1.9222,513.9454,2.9970,0.0681,0.1551,0.0099


In [63]:
dinner_model_f = blend_models(estimator_list=dinner_regression_best_models, fold=5, optimize='MAE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,55.3205,6516.5034,80.7249,0.6735,1.1347,0.1209
1,56.8867,6858.8998,82.8185,0.6709,1.0507,0.1104
2,56.2254,6787.6252,82.3870,0.5688,0.7098,0.1244
3,58.0156,7991.8682,89.3972,0.6159,1.1317,0.1251
4,58.2955,7486.8595,86.5266,0.5429,0.8732,0.1159
Mean,56.9487,7128.3512,84.3709,0.6144,0.9800,0.1193
SD,1.1072,536.1429,3.1480,0.0527,0.1651,0.0055


In [57]:
pred = predict_model(dinner_model_f)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,54.8764,5521.7926,74.3088,0.6559,0.6737,0.1223


## 테스트

### 테스트 데이터 예측

In [58]:
final_model_dinner = finalize_model(dinner_model_f)
prep_pipe_dinner = get_config('prep_pipe')
prep_pipe_dinner.steps.append(['trained_model', final_model_dinner])

In [59]:
pred_dinner = prep_pipe_dinner.predict(dinner_df_test_f)
submission.iloc[:,2] = pred_dinner
submission.head()

,일자,중식계,석식계
0,2021-01-27,984.525491,199.990676
1,2021-01-28,923.296760,426.967676
2,2021-01-29,541.073407,254.959411
3,2021-02-01,1262.244992,493.758240
4,2021-02-02,979.870525,449.174443


# 저장

In [99]:
import datetime
today = str(datetime.datetime.now().date()).replace("-","")
print("오늘 날짜 : " + today)

submission.to_csv(f'../submission/{today}_pycaret_(4-2).csv', index =False)

오늘 날짜 : 20210624
